# SheCodeAfrica Advanced Machine Learning Project

## Prediction of Rent Prices for Houses in Nigeria

The aim of this project is to
- Extract data from a Nigerian property website(Site used was propertypro.ng)
- Clean, Preprocess and Analyse the data extracted
- Build a machine learning model that predicts house rent price based on some details specifically location, price, number of rooms and bathrooms
- Deploy to a web app using flask

## Web Scraping

importing the needed libraries

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests

In [2]:
#Creating a driver
driver=webdriver.Chrome('C:/User/PC/Downloads/chromedriver_win32.exe')

C:\Users\PC\AppData\Local\Temp/ipykernel_24972/3246509272.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('C:/User/PC/Downloads/chromedriver_win32.exe')


There are multiple pages, about 370 so I created a while loop to make it easier extract data from all 370 at once

In [2]:
page = 1
while page != 370:
    url = 'https://www.propertypro.ng/property-for-rent/house/?search=&auto=&bedroom=&min_price=&max_price=&page={page}'
    driver.get(url)
    page = page + 1

NameError: name 'driver' is not defined

Created an emply list for the columns I am interested in

In [10]:
addresses = []
prices = []
details =[]

In [11]:
page = 1
while page != 370:
    url = 'https://www.propertypro.ng/property-for-rent/house/?search=&auto=&bedroom=&min_price=&max_price=&page={page}'
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    for element in soup.findAll('div', attrs={'class': 'single-room-sale'}):
        address = element.find('div', attrs={'class': 'single-room-text'})
        price = element.find('div', attrs={'class':'n50'})
        detail = element.find('div', attrs={'class': 'fur-areea'})
        
        if address and address.text:
            addresses.append(address.text)
        else:
            addresses.append('No display data')
        
        if price and price.text:
            prices.append(price.text)
        else:
            prices.append('No display data')
        
        if detail and detail.text:
            details.append(detail.text)
        else:
            details.append('No display data')
            
    page = page + 1

In [ ]:
addresses = []
prices = []
details =[]

Created a soup to extract the data, then an if statement so empty data does not return error while the scraper is parsing the site

In [12]:
content = driver.page_source
soup = BeautifulSoup(content, features='html.parser')
for element in soup.findAll('div', attrs={'class': 'single-room-sale'}):
    address = element.find('div', attrs={'class': 'single-room-text'})
    price = element.find('div', attrs={'class':'n50'})
    detail = element.find('div', attrs={'class': 'fur-areea'})
    
    if address and address.text:
        addresses.append(address.text)
    else:
        addresses.append('No display data')

    if price and price.text:
        prices.append(price.text)
    else:
        prices.append('No display data')
        
    if detail and detail.text:
        details.append(detail.text)
    else:
        details.append('No display data')
        

NameError: name 'driver' is not defined

Created a dataframe with the lists generated earlier, then saved it to a file

In [17]:
df = pd.DataFrame({'Address': addresses, 'Price': prices, 'Details': details})
df.to_csv('listings.csv', index=False, encoding='utf-8')

Checking the shape of the dataframe to know how much data was extracted

In [16]:
df.shape

(8118, 3)

## Data Cleaning

importing the csv file to start analysis

In [3]:
listings = pd.read_csv('listings.csv')

In [6]:
listings.head()

,Address,Price,Details
0,\n 4 BEDROOM HOUSE FOR RENT \nOld Ikoyi Ikoyi ...,"\n ₦ 14,000,000/year \nSponsored\n",\n4 beds\n4 baths\n5 Toilets\n
1,\n 4 BEDROOM HOUSE FOR RENT \nBanana Island Ik...,"\n ₦ 14,000,000/year \nPremium Gold\n",\n4 beds\n4 baths\n5 Toilets\n
2,\n 3 BEDROOM HOUSE FOR RENT \nOld Ikoyi Old Ik...,"\n ₦ 12,000,000 \nPremium Gold\n",\n3 beds\n3 baths\n4 Toilets\n
3,\n 2 BEDROOM HOUSE FOR RENT \nAmerican Interna...,"\n ₦ 2,000,000/year \nPremium Gold\n",\n2 beds\n2 baths\n3 Toilets\n
4,\n 4 BEDROOM HOUSE FOR RENT \nKolapo Ishola Gr...,"\n ₦ 2,500,000/year \nPremium Gold\n",\n4 beds\n4 baths\n5 Toilets\n


In [9]:
listings['Address']

0       \n 4 BEDROOM HOUSE FOR RENT \nOld Ikoyi Ikoyi ...
1       \n 4 BEDROOM HOUSE FOR RENT \nBanana Island Ik...
2       \n 3 BEDROOM HOUSE FOR RENT \nOld Ikoyi Old Ik...
3       \n 2 BEDROOM HOUSE FOR RENT \nAmerican Interna...
4       \n 4 BEDROOM HOUSE FOR RENT \nKolapo Ishola Gr...
                              ...                        
8113    \n 4 BEDROOM HOUSE FOR RENT \nOsapa London Lek...
8114    \n 4 BEDROOM HOUSE FOR RENT \nAjah Lagos\n\n ₦...
8115    \n 5 BEDROOM HOUSE FOR RENT \nIkota Lekki Lago...
8116    \n 4 BEDROOM HOUSE FOR RENT \nLekki Lagos\n\n ...
8117    \n 4 BEDROOM HOUSE FOR RENT \nLekki Phase 1 Le...
Name: Address, Length: 8118, dtype: object